# iBP e Algebra Geométrica Conforme

Usando a Algebra Geométrica Conforme, vamos ver como gerar árvore de busca do iBP, sem fazer amostragem.

In [2]:
from cowboy import *
import math

O ponto no modelo euclidiano pode ser representado por $$X_i = \alpha_1 e_1 + \alpha_2 e_2 + \alpha_3 e_3,$$ onde $e_1, e_2, e_3$ são as bases do espaço Euclidiano, e os pontos $X_i$ representam o centro do atomo de uma molecula de proteina.

Iremos utilizar o modelo conforme da Algebra Geométrica para representarmos as circufenrencias, planos e ponto.
Nesse modelo os pontos são representados por $$x_i = e_0 + X_i + \frac{1}{2}\parallel X_i \parallel^2e_{\infty},$$ onde $X_i$ representa o ponto no modelo euclidiano. Assim teremos, $x_1, x_2, x_3, x_4$, os quatro primeiros pontos que representam o centro do quatro primeiros atomos da molecula. Para o primeiro ponto $x_1$ podemos fixar ele na origem do nosso sistema de coordenadas.


In [3]:
e1 = MultiVetor([[1, 2**0]])
e2 = MultiVetor([[1, 2**1]])
e3 = MultiVetor([[1, 2**2]])
emais = MultiVetor([[1, 2**3]])
emenos = MultiVetor([[1, 2**4]])
e0 = 0.5 * (emenos + emais)
einf = emenos - emais

In [4]:
def ponto_conforme(x):
    x_conforme = e0 + x + 0.5*(x*x)*einf
    return x_conforme

In [5]:
# Modelo Euclidiano
x2 = -1*e1
x3 = -1.5*e1 + (math.sqrt(3)/2)*e2
x4 = -1.311*e1 + 1.552*e2 + 0.702*e3

# Modelo Conforme
x1 = e0
x2 = e0 + x2 + 0.5*(x2*x2)*einf
x3 = e0 + x3 + 0.5*(x3*x3)*einf
x4 = e0 + x4 + 0.5*(x4*x4)*einf

Neste problema são conhecidas as distancias entre atomos que fazem ligação, entretanto as distancias entre atomos que não fazem ligações são fornenecidas pelo experimento de Ressonancia Magnetica Nuclear, onde geralmente essas distancias são imprecisas. Para o nosso exemplo temos as distancias $$d_{i-1, i} = 1, \\ d_{i-2, i} = \sqrt{3}, \\ d_{1, 4} = 2.15, \\ d_{2, 5} \in [2.2, 2.6], \\ d_{1, 5} \in [2.45, 2.55]$$

In [6]:
raio = {'i-1': 1, 'i-2': math.sqrt(3), '1,4': 2.15, '2,5': [2.2, 2.6], '1,5': [2.45, 2.55]}

Uma esfera no $\mathbb{R}^3$, centrada em C, e com raio r, pode ser representada no $\mathbb{R}^{4,1}$ como:
$$S = C - \frac{r^2}{2}e_{\infty} $$

In [7]:
def esfera(x, raio):
    #transforma para um ponto no modelo conforme
    conforme = False
    for blade in x.componentes:
        if blade[1] == 2**4 or blade[1] == 2**3:
           conforme = True 

    if not conforme:
        x = e0 + x + 0.5*(x*x)*einf
    
    return x - (0.5*(raio**2)*einf)

Para realizar a extração dos pontos iremos fazer o produto externo entre as esferas $S_{i-1}, S_{i-2}, S_{i-3}$. Assim temos que o par de pontos pode ser obtido por: 
$$pp = S_{i-1} \wedge S_{i-2} \wedge S_{i-3},$$ e o seu dual por: 
$$pp^{*} = S_{i-1} \wedge S_{i-2} \wedge S_{i-3} \rfloor (-1)\cdot(e_0 \wedge e_1 \wedge e_2 \wedge e_3 \wedge e_{\infty})$$
e a extração do ponto é feita por:
$$x_i = \frac{\sqrt{|pp^* \rfloor pp^*|} + pp^*}{(-1)\cdot(e_\infty \rfloor pp^*)}$$
caso $\Delta = pp^* \rfloor pp^* < 0$ não temos a intersecção dessas esferas, caso $\Delta = 0$ a esfera intersecta em um ponto e caso $\Delta > 0$ ela intersecta em mais de um ponto

In [8]:
def analise(mult):
    mult_new = list()
    soma = 0

    for blade in mult.componentes:
        if abs(blade[0]) > 0.0000001:
            mult_new.append(blade)
        if blade[1] == 2**3:
            soma = soma + blade[0]
        elif blade[1] == 2**4:
            soma = soma + blade[0]

    mult_new = MultiVetor(mult_new)

    mult_new = (1/soma)*mult_new

    return mult_new

def extracao_ponto(s1, s2, s3):
    i = e0^e1^e2^e3^einf
    pp = s1^s2^s3
    dualpp = pp<<(-1*i)

    # EXTRACAO
    x4_a = (1*math.sqrt(float(abs(dualpp<<dualpp))) + dualpp) / -1*(einf<<dualpp)
    x4_b = (-1*math.sqrt(float(abs(dualpp<<dualpp))) + dualpp) / -1*(einf<<dualpp)

    return analise(x4_a), analise(x4_b)

In [9]:
a, b = extracao_ponto(esfera(x2, raio['2,5'][0]), esfera(x3, raio['i-2']), esfera(x4, raio['i-1']))
print(f'a = {a} \nb = {b} \n')

c, d = extracao_ponto(esfera(x2, raio['2,5'][1]), esfera(x3, raio['i-2']), esfera(x4, raio['i-1']))
print(f'c = {c} \nd = {d}')

a = -1.5008576783864445*e1 + 1.350504449056432*e2 + 1.66291290183224*e3 - 2.9208576783864446*e4 + 3.9208576783864446*e5 
b = -0.409066047772618*e1 + 1.980850640890306*e2 + 0.7530124850156146*e3 - 1.8290660477726186*e4 + 2.829066047772619*e5 

c = -2.045056613979804*e1 + 2.1448235639429822*e2 + 1.0332417689640665*e3 - 4.425056613979805*e4 + 5.425056613979804*e5 
d = -1.3863948809680542*e1 + 2.525102092802222*e2 + 0.4843123133759116*e3 - 3.7663948809680554*e4 + 4.766394880968055*e5


In [10]:
z = (x3^x4^einf)<<(-1*(e0^e1^e2^e3^einf))

r = rotor(z, 1.453)
x5_a = r*a*inverso(r)

r = rotor(z, -1.453)
x5_b = r*b*inverso(r)

print(f" X_5(1.453) = {analise(x5_a)} \n X_5(-1.453) = {analise(x5_b)}")

 X_5(1.453) = -0.4545103617082674*e1 + 1.5184688468969887*e2 + 1.217073937673492*e3 - 1.496798138830456*e4 + 2.496798138830456*e5 
 X_5(-1.453) = -0.8567679105762847*e1 + 1.2862253427744788*e2 + 1.5523158475028107*e3 - 1.8990556876984739*e4 + 2.899055687698474*e5


Assim nos temos os dois rotores associado aos arcos de possiveis soluções para $X_{5}$, como nos sabemos a distancia de $d_{1, 5} \in [2.45, 2.55]$, nos podemos fazer a extração dos pontos: $$pp = S_{1, 5} \wedge S_{3} \wedge S_{4},$$ onde $S_{1, 5}$ é a esfera de centro $X_{1}$ e raio $d_{1, 5} \in [2.45, 2.55]$, como nos temos um intervalo de distancia, iremos pegar os extremos desse invervalo assim temos $\underline{d_{1,5}} = min(d_{1, 5})$ e $\overline{d_{1,5}}  = max(d_{1, 5})$

In [15]:
P_a, P_b = extracao_ponto(esfera(x3, raio['i-2']), esfera(x4, raio['i-1']), esfera(x1, raio['1,5'][0]))
P_c, P_d = extracao_ponto(esfera(x3, raio['i-2']), esfera(x4, raio['i-1']), esfera(x1, raio['1,5'][1]))

print(f'P_a = {analise(P_a)} \nP_b = {analise(P_b)}\n')
print(f'P_c = {analise(P_c)} \nP_d = {analise(P_d)}\n')

P_a = -1.2599837919047157*e1 + 1.283189046518469*e2 + 1.6638409524448043*e3 - 2.5012499999999998*e4 + 3.5012499999999998*e5 
P_b = -0.6735069215271265*e1 + 2.29899678347644*e2 + 0.5133246694052717*e3 - 2.5012499999999998*e4 + 3.5012499999999998*e5

P_c = -1.4066508602468961*e1 + 1.3178293669474492*e2 + 1.6694786961736159*e3 - 2.7512500000000006*e4 + 3.7512500000000006*e5 
P_d = -0.7952701420775019*e1 + 2.376771833584789*e2 + 0.4701074900467275*e3 - 2.7512500000000006*e4 + 3.7512500000000006*e5

